# Dimensionality Reduction

In [39]:
%pip install -U scikit-learn


ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\se99a\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "c:\Users\se99a\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "c:\Users\se99a\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "c:\Users\se99a\anaconda3\lib\http\client.py", line 462, in read
    n = self.readinto(b)
  File "c:\Users\se99a\anaconda3\lib\http\client.py", line 506, in readinto
    n = self.fp.readinto(b)
  File "c:\Users\se99a\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "c:\Users\se99a\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "c:\Users\se99a\anaconda3\lib\ssl.py", line 1099, in read
    return self._sslobj.read(len, buffer)
Con

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Union

from sklearn.metrics import confusion_matrix
from scipy.stats import norm

In [12]:
# Performance evaluation function for Classification --------------------------
def perf_eval_clf(y_true: Union[np.array,list], y_pred: Union[np.array,list])->list:
    """
    Input:
    y_true : ndarray of shape (n_samples,)
    y_pred : ndarray of shape (n_samples,)
    Return:
    [TPR,Precision,TNR, Accuracy, BCR, F1]: scores of classification in a list
    """
    cm = confusion_matrix(y_true = y_true, y_pred = y_pred)
    print(f"Confusion Matrix:\n{cm}")
    tn, fp, fn, tp = cm.ravel()
    TPR = round(tp/(tp+fn),3) #==recall
    PRE = round(tp/(tp+fp),3)
    TNR = round(tn/(tn+fp),3)
    ACC = round((tp+tn)/(tp+tn+fp+fn), 3)
    BCR = round(np.sqrt([TPR*TNR])[0],3)
    F1 = round(2*TPR*PRE/(TPR+PRE),3)
    return [TPR, PRE, TNR, ACC, BCR, F1]
    

In [30]:
def display_result(result_dict: dict, metric_names: list = ['TPR(Recall)','Precision','TNR','ACC','BCR','F1']):
    
    result_df = pd.DataFrame(result_dict).T
    result_df.index = list(result_dict.keys())
    result_df.columns = metric_names
    
    display(result_df)
    
result_dict = {}

In [27]:
def display_coefs(model, feature_names, name: str) -> None:
    """
    input:
    model: fitted model with given variables
    name: name of the model
    """
    features = ['Constant']+feature_names if model.intercept_ else feature_names
    coefs = [model.intercept_[0]]+model.coef_[0].tolist() if model.intercept_ else model.coef_[0].tolist()
    print(f'Model : {name}')
    display(pd.DataFrame({'features':features,'coefs':coefs}))

In [15]:
from sklearn.preprocessing import scale
ploan = pd.read_csv("Personal Loan.csv")
# Indices for the activated input variables
#input_idx = set(range(len(ploan.columns.tolist())))-set([0,4,9])
#input_idx = list(input_idx)
input_idx = [1,2,3,5,6,7,8,10,11,12,13]
target_idx = 9
ploan_input = ploan.iloc[:,input_idx]
ploan_input_scaled = scale(ploan_input,axis =0, with_mean=True, with_std=True)
ploan_target = ploan.iloc[:,target_idx]
ploan_data = pd.concat((ploan_input,ploan_target),axis=1)

In [16]:
ploan_trn = ploan_data.iloc[:1500,:]
ploan_tst = ploan_data.iloc[1500:,:]
x_trn, y_trn = ploan_trn.iloc[:,:-1],ploan_trn.iloc[:,-1]
x_tst, y_tst = ploan_tst.iloc[:,:-1],ploan_tst.iloc[:,-1]
# feature names
feature_names = list(x_trn.columns)

In [17]:
def logit_pvalue(model,x,feature_names,cls=0):
    p1 = model.predict_proba(x)
    n1 = len(p1)
    m1 = len(model.coef_[cls]) + 1
    if len(model.intercept_)>1:
        feature_names=['constant'] + feature_names
        coefs = np.concatenate([np.array(model.intercept_[cls].reshape(1,)), model.coef_[cls]])
    elif len(model.intercept_)==1:
        feature_names=['constant'] + feature_names
        coefs = np.concatenate([model.intercept_, model.coef_[0]])
    else:
        coefs = model.coef_[cls]
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    answ = np.zeros((m1, m1))
    for i in range(n1):
        answ = answ + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p1[i,1] * p1[i, 0]
    vcov = np.linalg.inv(np.matrix(answ))
    se = np.sqrt(np.diag(vcov))
    t1 = coefs/se
    p1 = (1 - norm.cdf(abs(t1))) * 2
    pvalue_dict = {key:round(value,4) for key,value in zip(feature_names,p1)}
    display(pd.DataFrame(pvalue_dict,columns=pvalue_dict.keys(),index=['P-value']).T)
    return pvalue_dict

## 1. All variables

In [32]:
from sklearn.linear_model import LogisticRegression
seed = 1234
full_config = {
    'penalty': 'none',
    'fit_intercept':True,
    'max_iter':1e+5,
    'solver':'saga',
    'random_state':seed,
    'n_jobs':-1
}

#solving name of the selected features
features_dict = {}
features_dict['All Variables'] = ['constant']+feature_names

full_model = LogisticRegression(**full_config).fit(x_trn,y_trn)
logit_pvalue(full_model, x_trn, feature_names)


y_pred = full_model.predict(x_tst)
result_dict['All variables'] = perf_eval_clf(y_tst,y_pred)
display_coefs(full_model, feature_names, 'All_variables')
display_result(result_dict,)

,P-value
constant,0.9835
Age,0.0008
Experience,0.0007
Income,0.0000
Family,0.0000
CCAvg,0.0553
Education,0.0000
Mortgage,0.8055
Securities Account,0.9961
CD Account,0.3465


Confusion Matrix:
[[876  20]
 [ 47  57]]
Model : All_variables


,features,coefs
0,Constant,-0.063633
1,Age,-0.411730
2,Experience,0.418377
3,Income,0.042928
4,Family,0.557976
5,CCAvg,0.107522
6,Education,0.977913
7,Mortgage,0.000215
8,Securities Account,-0.001934
9,CD Account,0.413303


,TPR(Recall),Precision,TNR,ACC,BCR,F1
All variables,0.548,0.74,0.978,0.933,0.732,0.63


## 2. Forward Selection

In [40]:
from sklearn.feature_selection import SequentialFeatureSelector as SFS
full_config = {
'penalty':'none',
'fit_intercept':True,
'max_iter':1e+5,
'solver':'saga',
'random_state':seed,
'n_jobs':-1
}

full_model = LogisticRegression(**full_config)

# fitting SequentialFeatureSelector
forward_config = {
    'estimator':full_model,
    'n_features_to_select':'auto', #변수개수 지정
    'tol':1e-4, #성능차이 이정도면 stop
    'direction':'forward',
    'scoring':'f1',
    'cv':5, #cross validation
    'n_jobs':-1
    }

forward_selection = SFS(**forward_config).fit(x_trn,y_trn)
features_dict['Forward Selection']=forward_features = forward_selection.get_feature_names_out()
print(forward_features)

TypeError: __init__() got an unexpected keyword argument 'tol'